In [1]:
from model import Network
from data import load_cifar10_dataloaders, WhiteningTransformation
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from criterion import ReconstructImageFromFCLoss
from metrics import psnr_matching, psnr
import optuna

In [2]:
def objective(trial):
    # load to model
    model_config = './model_config/fc1_cocktail_party_cifar10_instance.json'
    checkpoint_path = './checkpoints/122123_fc1_cocktail_party_cifar10_pretraining_wout_bias_wout_normalization.pth'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = Network(model_config)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    
    # get val loader
    batch_size = 4
    data_path = './data'
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    _, val_dataloader = load_cifar10_dataloaders(data_path, batch_size, transform, batch_sampler=True)
    selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
    selected_val_batch_data = selected_val_batch_data.to(device)
    selected_val_batch_label = selected_val_batch_label.to(device)
    
    # receiving gradients
    model.zero_grad()
    criterion = nn.CrossEntropyLoss()
    output = model(selected_val_batch_data.reshape(batch_size, -1))
    loss = criterion(output, selected_val_batch_label)
    loss.backward()
    gradient_of_layers = []
    for param in model.parameters():
        gradient_of_layers.append(param.grad.data.clone().to('cpu'))
    
    whitening_transform = WhiteningTransformation()
    whitened_gradient = torch.from_numpy(whitening_transform.transform(gradient_of_layers[0].detach().numpy().T)).to(torch.float32).T
    whitened_gradient = whitened_gradient.to(device)
    
    # criterion output testing
    unmixing_matrix = torch.rand((selected_val_batch_data.size(0), gradient_of_layers[0].size(0)), requires_grad=True, device=device)
    t_param = trial.suggest_float('t_param', 0.00001, 10)
    total_variance_loss_param = trial.suggest_float('total_variance_loss_param', 0.00001, 10)
    mutual_independence_loss_param = trial.suggest_float('mutual_independence_loss_param', 0.00001, 10)
    reconstruction_loss = ReconstructImageFromFCLoss(32, 32, 3, t_param, total_variance_loss_param, mutual_independence_loss_param)
    optimizer = torch.optim.Adam([unmixing_matrix])
    
    for iter_idx in range(25000):
        optimizer.zero_grad()
        # out_score, non_gaussianity_score, total_variance_score, mutual_independence_score
        loss, _, _, _ = reconstruction_loss(unmixing_matrix, whitened_gradient)
        loss.backward()
        optimizer.step()
        
        if (iter_idx + 1) % 1000 == 0 or iter_idx == 0:
            print('loss: {}'.format(loss.item()))
    
    with torch.no_grad():
        unmixing_matrix = unmixing_matrix.detach().to('cpu')
        whitened_gradient = whitened_gradient.detach().to('cpu')
        estimated_img = unmixing_matrix @ whitened_gradient
        matches = psnr_matching(estimated_img.reshape(estimated_img.shape[0], 3, 32, 32), selected_val_batch_data.detach().to('cpu'))
        matches = torch.stack(matches, dim=0)
        return psnr(estimated_img.reshape(estimated_img.shape[0], 3, 32, 32)[matches[:, 0]], selected_val_batch_data.detach().to('cpu')[matches[:, 1]])

In [3]:
num_of_trials = 100
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=num_of_trials)

[I 2023-12-24 01:12:55,937] A new study created in memory with name: no-name-6b7554fe-6374-4016-92fb-f51a7394984e


Files already downloaded and verified
Files already downloaded and verified
loss: 157.9277801513672
loss: 1.0608521699905396
loss: 0.3277541697025299
loss: 0.18397720158100128
loss: 0.13981521129608154
loss: 0.1302921622991562
loss: 0.1290368139743805
loss: 0.1272035539150238
loss: 0.1246122419834137
loss: 0.12112525850534439
loss: 0.11664848774671555
loss: 0.11092035472393036
loss: 0.10369201004505157
loss: 0.09524835646152496
loss: 0.08755199611186981
loss: 0.08258409053087234
loss: 0.0798039361834526
loss: 0.07757095992565155
loss: 0.07589119672775269
loss: 0.0753006562590599
loss: 0.07444910705089569
loss: 0.07308124005794525
loss: 0.07264288514852524
loss: 0.07249875366687775
loss: 0.07186336070299149


[I 2023-12-24 01:13:37,648] Trial 0 finished with value: 7.590768337249756 and parameters: {'t_param': 9.516416341108215, 'total_variance_loss_param': 1.2129304184982075, 'mutual_independence_loss_param': 0.06582561477588714}. Best is trial 0 with value: 7.590768337249756.


loss: 0.07148733735084534
Files already downloaded and verified
Files already downloaded and verified
loss: 43.5206184387207
loss: 8.814992904663086
loss: 8.801575660705566
loss: 8.787923812866211
loss: 8.768777847290039
loss: 8.750848770141602
loss: 8.736002922058105
loss: 8.724018096923828
loss: 8.70984935760498
loss: 8.702596664428711
loss: 8.691622734069824
loss: 8.685279846191406
loss: 8.680564880371094
loss: 8.667215347290039
loss: 8.65923023223877
loss: 8.65715503692627
loss: 8.658051490783691
loss: 8.649267196655273
loss: 8.641928672790527
loss: 8.635754585266113
loss: 8.629660606384277
loss: 8.63213062286377
loss: 8.627727508544922
loss: 8.625678062438965
loss: 8.62905216217041


[I 2023-12-24 01:14:18,076] Trial 1 finished with value: 7.601733207702637 and parameters: {'t_param': 2.264734444802563, 'total_variance_loss_param': 5.689807252474181, 'mutual_independence_loss_param': 8.498819276007811}. Best is trial 1 with value: 7.601733207702637.


...

loss: 9.899603843688965
Files already downloaded and verified
Files already downloaded and verified
loss: 109.33760070800781
loss: 8.840760231018066
loss: 8.843560218811035
loss: 8.842950820922852
loss: 8.840723991394043
loss: 8.845173835754395
loss: 8.839044570922852
loss: 8.839646339416504
loss: 8.841141700744629
loss: 8.839400291442871
loss: 8.8394775390625
loss: 8.837273597717285
loss: 8.83795166015625
loss: 8.836605072021484
loss: 8.834370613098145
loss: 8.833368301391602
loss: 8.832595825195312
loss: 8.832457542419434
loss: 8.831170082092285
loss: 8.829472541809082
loss: 8.82799243927002
loss: 8.83256721496582
loss: 8.828153610229492
loss: 8.827397346496582
loss: 8.826132774353027


[I 2023-12-24 02:13:45,559] Trial 98 finished with value: 7.597008228302002 and parameters: {'t_param': 3.311970819942578, 'total_variance_loss_param': 1.7305478820616187, 'mutual_independence_loss_param': 8.737726516242232}. Best is trial 43 with value: 7.664186954498291.


loss: 8.825933456420898
Files already downloaded and verified
Files already downloaded and verified
loss: 96.30418395996094
loss: 3.38784122467041
loss: 3.240046977996826
loss: 3.2391762733459473
loss: 3.2400074005126953
loss: 3.2399139404296875
loss: 3.2405107021331787
loss: 3.239112377166748
loss: 3.238053560256958
loss: 3.2394590377807617
loss: 3.2374041080474854
loss: 3.2376813888549805
loss: 3.236264944076538
loss: 3.236229419708252
loss: 3.2348854541778564
loss: 3.2346251010894775
loss: 3.233991861343384
loss: 3.2332677841186523
loss: 3.2325499057769775
loss: 3.231790065765381
loss: 3.233441114425659
loss: 3.2311360836029053
loss: 3.231088876724243
loss: 3.230602741241455
loss: 3.2287814617156982


[I 2023-12-24 02:14:23,195] Trial 99 finished with value: 7.57021427154541 and parameters: {'t_param': 4.272311109060551, 'total_variance_loss_param': 0.9730675100971189, 'mutual_independence_loss_param': 3.1800057070433025}. Best is trial 43 with value: 7.664186954498291.


loss: 3.2301745414733887
